In [13]:
def inf_data ( data ) : 
    data2 = data.split("/")    ##[0] for inf 1 , [-1] for inf 2
    data3 = data2[1].split("-") ## [-1] for inf1 
    out_data_0 = float(data2[0])
    out_data_1 = float(data3[-1].split(")")[0])
    out_data_2 = float(data2[-1].split(")")[0])
    
    out_data = [out_data_0, out_data_1, out_data_2]
    return out_data




In [49]:
data = "1/(1+exp(((v -(-20.800))/(-8.100))))"
out_data = inf_data ( data )
out_data

ValueError: could not convert string to float: '(-8.100'

In [43]:
val = 80
import numpy as np 

#val = (potential * gate_constants[0]) + gate_constants[1]
gate_tau =  np.clip(val, -1000 , 80 )
gate_tau

80

In [11]:
inf_Kv__default
tau_Kv__default
tau_Kv__clipping
tau_Kv__const

{'m': {'constants_inf': [1.0, 30.5, 11.39343],
  'constants_tau': [30.0, 76.56, 26.1479],
  '_inf': 0.0049172433592656745,
  '_tau': 0.5011113839540371,
  '_pow': 1,
  'inf_func': <function __main__.inf_Kv1_1(potential, gate_constants)>,
  'tau_func': <function __main__.tau_Kv1_1(potential, gate_constants)>},
 'h': {'constants_inf': [1.0, 30.0, 27.393],
  'constants_tau': [15000.0, 160.56, 100.0],
  '_inf': 0.1006212664041962,
  '_tau': 1942.1396507245397,
  '_pow': 2,
  'inf_func': <function __main__.inf_Kv1_1(potential, gate_constants)>,
  'tau_func': <function __main__.tau_Kv1_1(potential, gate_constants)>}}

In [ ]:
def inf_Kv__default( potential ,gate_constants  ) :
    gate_inf = gate_constants[0] / ( 1 + np.exp( (potential - (gate_constants[1] ) ) / gate_constants[2])    )
#1.0000/(1+ exp((v - -30.5000)/-11.3943))
    return gate_inf
def tau_Kv__default(potential, gate_constants):
#1.0000/(1+ exp((v - -30.0000)/27.3943)) 
    gate_tau = gate_constants[0] / ( 1 + np.exp( (potential - (-gate_constants[1] ) ) / gate_constants[2])    )
    return gate_tau

In [116]:
import numpy as np 


def calc_gate_inf(alpha_gate: np.array, beta_gate: np.array) -> np.array:
    gate_inf = alpha_gate / (alpha_gate + beta_gate)
    return gate_inf

#eq. 2.1.4, 2.2.4 , 3.4 , 4.2.6
def calc_tau_gate(alpha_gate: np.array, beta_gate: np.array) -> np.array: 
    tau = 1. / (alpha_gate + beta_gate) 
    return tau

class Kv_x :
    def __init__(self,
                 potential : float , 
                 reversal_potential :float,
                 conductance : float ,
                 T : float ,
                 animal : str
                 ) :
        self.potential = potential
        self.reversal_potential = reversal_potential
        self.gates = {} 
        self.conductance = conductance
        self.T = T #temp in C
        self.animal = animal 
    #change to have alpha beta per gate... 
    def add_gate(self, gate_name : str, 
                 gate_constants_inf : np.array =None,
                 gate_constants_tau : np.array =None,
                 gate_power : int =1 ,
                 inf_function =None,
                 tau_function =None,
                 alpha_parms : np.array =None,
                 beta_parms : np.array=None,
                 alpha_function=None,
                 beta_function =None
                  ) :
        if gate_name in  self.gates.keys() :
             self.gates[gate_name] = {} #clear gate if trying to add to it again 

        tmp_dict= {
            "constants_inf" :gate_constants_inf,
            "constants_tau" : gate_constants_tau,
            "_pow" : gate_power,
            "inf_func" : inf_function ,
            "tau_func" : tau_function,
            "alpha_func" : alpha_function,
            "beta_func" : beta_function,
            "alpha_parms" :alpha_parms,
            "beta_parms" : beta_parms 
            }
        self.gates[gate_name]  = {k : v for k,v  in tmp_dict.items() if v is not None }
        
        #if inf_constants + tau constants 
        if "constants_inf" in self.gates[gate_name] :
            self.gates[gate_name]["_inf" ] = inf_function(self.potential, gate_constants_inf)
        if "constants_tau" in self.gates[gate_name] :
            self.gates[gate_name]["_tau" ] = tau_function(self.potential, gate_constants_inf)
        # if alpha and beta 
        if "alpha_parms" and "beta_parms" in self.gates[gate_name] :
            alpha = self.gates[gate_name]["alpha_func"](self.potential, self.gates[gate_name]["alpha_parms"])
            beta =  self.gates[gate_name]["beta_func"](self.potential, self.gates[gate_name]["beta_parms"])
            self.gates[gate_name]["_tau" ] = calc_tau_gate(alpha, beta)
            self.gates[gate_name]["_inf" ] = calc_gate_inf(alpha, beta)
        
# "_inf" : inf_function(self.potential, gate_constants_inf  ), 
# "_tau" : tau_function(self.potential , gate_constants_tau ),

In [119]:
Kv_1_1 = Kv_x(V, -65, 0.00001, 24, "rat")

Kv_1_1.add_gate("m", 
                gate_constants_inf = [1.0, -30.5, -11.39343], 
                gate_constants_tau = [30.0, -76.56, 26.1479], 
                gate_power = 1,
                inf_function = inf_Kv__default, 
                tau_function = tau_Kv__default)

In [120]:
Kv_1_1.gates

{'m': {'constants_inf': [1.0, -30.5, -11.39343],
  'constants_tau': [30.0, -76.56, 26.1479],
  '_pow': 1,
  'inf_func': <function __main__.inf_Kv__default(potential, gate_constants)>,
  'tau_func': <function __main__.tau_Kv__default(potential, gate_constants)>,
  '_inf': 0.9950827566407344,
  '_tau': 0.4890305252650619}}

In [54]:
def inf_Kv__default( potential ,gate_constants  ) :
    gate_inf = gate_constants[0] / ( 1 + np.exp( (potential - (gate_constants[1] ) ) / gate_constants[2])    )
#1.0000/(1+ exp((v - -30.5000)/-11.3943))
    return gate_inf
def tau_Kv__default(potential, gate_constants):
#1.0000/(1+ exp((v - -30.0000)/27.3943)) 
    gate_tau = gate_constants[0] / ( 1 + np.exp( (potential - (-gate_constants[1] ) ) / gate_constants[2])    )
    return gate_tau

In [111]:
Kv_1_1.add_gate("m" , gate_constants_tau = [30.0, -76.56, 26.1479] ) 
# , 
#                 gate_constants_inf = [1.0, -30.5, -11.39343], 
#                 gate_constants_tau = [30.0, -76.56, 26.1479], 
#                 gate_power = 1,
#                 beta_parms = inf_Kv__default, 
#                 tau_function = tau_Kv__default)

In [112]:
Kv_1_1.gates

{'m': {'constants_tau': [30.0, -76.56, 26.1479], '_pow': 1}}